In [1]:
import pandas as pd
import numpy as np
import json
import os
import re
from pyathena import connect
from pyathena.pandas.util import as_pandas
from pyathena.pandas.cursor import PandasCursor

In [128]:
# Load client data
client = "länsförsäkringar"
client_df = pd.read_excel(f"../datasets/{client.title()}.xlsx", converters={"Imps": int, "Placement Id": int})
#client_df["Viewability Rate %"] = client_df["Viewability Rate %"].str.replace(",", ".").astype(float)
client_placement_ids = list(client_df["Placement Id"].unique())

In [3]:
# Fetch impressions using PyAthena library
cursor = connect(s3_staging_dir="s3://aws-athena-query-results-094611745175-eu-west-1/",
                 region_name="eu-west-1", profile_name="atexprodadminsso", cursor_class=PandasCursor).cursor()

impressions_df = cursor.execute(f'''
select 
    "impression_model"."id", 
    "hostname", 
    "placement_ids", 
    "placement_ids_chosen", 
    "total_fixation_duration", 
    "channel", 
    "ad_technical_format", 
    "is_fixated", 
    "gaze_valid", 
    "is_iab_inview", 
    "exist_viewable_1_s_threshold_50", 
    "exist_viewable_2_s_threshold_50", 
    "impression_model"."part_month", 
    CONCAT(cast("ad_width_chosen" as VARCHAR), 'x', cast("ad_width_chosen" as VARCHAR)) as size,
    "groupm_more_brands"."chosen_brand" as "chosen_brand"
from "prod_attentionpanel_com_eu_west_1"."impression_model"
join "groupm-brand-batch"."groupm_more_brands" on impression_model.id = groupm_more_brands.id
where impression_model.part_year = '2023' and impression_model.part_month in ('01', '02', '03', '04', '05')
''').as_pandas()

In [129]:
impressions_df = pd.read_csv("../datasets/impressions/2023/impressions_01_05_brands.csv")
impressions_df

,id,hostname,beacon_ids,first_event_at_iso,ad_width_chosen,ad_height_chosen,placement_ids,placement_ids_chosen,total_fixation_duration,channel,beacon_type_chosen,ad_technical_format,is_fixated,gaze_valid,is_iab_inview,exist_viewable_1_s_threshold_50,exist_viewable_2_s_threshold_50,part_month,part_year,chosen_brand
0,a36a0f9a-1a67-4c83-a313-5d2899e1d262,classic.prisjakt.nu,"[""6ac7b3b3-19be-40eb-b46e-48397bcdd651""]",2023-02-19T05:08:57.685Z,285,230,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,99,display,ad/doubleclick.net-iframe-v1,display,True,True,True,True,True,2,2023,netonnet
1,d8b122f0-a707-40cd-ba94-79c775cba07e,cointiply.com,"[""2a579515-79ba-44e6-a9a8-66069bcea6cd""]",2023-02-19T05:45:15.915Z,728,90,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/criteo.com-iframe-v1,display,False,False,False,False,False,2,2023,samsung
2,6da9d257-1d33-4c38-b043-e009f25ba3b9,wrestlezone.com,"[""34802d95-f215-4591-8e2f-32e9854e66f5"",""b2b35...",2023-02-19T05:26:44.431Z,250,480,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/doubleclick-img-link-google-v1,display,False,False,True,True,True,2,2023,sodastream
3,21ae539c-b0f2-4fbf-8c43-4561e70c8cc9,placera.se,"[""25bfdb27-cb13-4104-95be-ed4d4f8215a5""]",2023-01-31T16:09:35.295Z,980,240,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/doubleclick.net-iframe-v1,display,False,False,True,True,True,1,2023,telia
4,101caa66-f19c-40dd-9d83-28719ce0dcac,facebook.com,"[""72ffbcd9-3d94-47dc-9ba4-cc830295fce0""]",2023-01-31T16:15:26.926Z,285,230,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,social,ad/facebook.com-sidebar-ads-v1,display,False,True,False,False,False,1,2023,bmw
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1305299,692dffbd-39b3-4401-8348-9c108196e311,youtube.com,"[""60d26c03-9a30-4ec9-bb6f-df0143f2e9f4""]",2023-04-12T16:59:26.886Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,False,False,False,False,4,2023,skoda
1305300,089dedfa-2e9e-479a-b684-267d6207ebdd,youtube.com,"[""31d74e22-6724-4fbd-88b1-17d21b592abe""]",2023-04-12T16:34:37.898Z,180,150,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-sidebar-featured-v1,display,False,False,False,False,False,4,2023,tui
1305301,9a4e4567-6ee1-41b2-a0b7-d3c7ee7a1082,youtube.com,"[""6eb590de-f5e8-4324-b4af-d8929813486f"",""e0427...",2023-04-12T16:27:08.314Z,468,120,"{""tag_id"":[],""bam_ad_slot"":["""","""",""""]}",NaN,0,display,ad/youtube.com-watch-companion-v1,display,False,False,False,False,False,4,2023,cmore
1305302,aa59325a-8e58-4e05-8b11-6d264dd1f20e,svd.se,"[""1d46f1a8-801a-4c8c-9d58-69e23fd6ae97"",""2ff3c...",2023-04-12T16:58:13.874Z,1180,240,"{""tag_id"":[""19499726""],""bam_ad_slot"":["""","""",""""]}",NaN,1234,display,ad/newssites-banners-v1,display,True,True,True,True,True,4,2023,amex


In [130]:
def extract_pid(placement_id):
    placement_id = json.loads(placement_id)
    bam_ad_slots = placement_id["bam_ad_slot"]
    final_bam_ad_slots = [bas for bas in bam_ad_slots if bas.strip() != '']
    try:
        ret = placement_id['tag_id'][0]
    except:
        return
    ret += ','.join([bas for bas in final_bam_ad_slots])
    return ret

def pid_type(pid):
    try:
        int(pid)
        return 'int'
    except:
        return 'str'

impressions_df['pid'] = impressions_df['placement_ids'].apply(extract_pid)
impressions_df['pid_type'] = impressions_df['pid'].apply(pid_type)
impressions_df = impressions_df[impressions_df["pid_type"] == 'int']
impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))

/tmp/ipykernel_11810/3375560408.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  impressions_df['pid'] = impressions_df['pid'].apply(lambda pid: int(pid))


In [131]:
# Get local predictions and store them in a map
def _get_pred_map():
    pred_map = dict()
    _year = 2023
    _range1 = 1
    _range2 = 6
    for i in range(_range1, _range2):
        if i < 10:
            i = f'0{i}'
        for day in os.listdir(f'../predictions/{_year}/{i}'):
            for h in os.listdir(f'../predictions/{_year}/{i}/{day}'):
                if not h.endswith('.ndjson'):
                    continue
                with open(f'../predictions/{_year}/{i}/{day}/{h}')as f:
                    for line in f.readlines():
                        json_line = json.loads(line)
                        id_ = json_line['id']
                        pred_map[id_] = json_line['prediction']
    return pred_map
                            
pred_map = _get_pred_map()

In [155]:
# Filter impressions based on tier
### TIER 1 ###
# This is the most accurate data we have. This is "campaign data", meaning correct brand and timeframe
### TIER 2 ###
# We might not have enough campaign data (tier 1). Tier 2 data is brand data, for a longer period (roughly 6 months)
### TIER 3 ###
# This is the least accurate data for a campaign. It ignore brand/timeframe and only checks Placement ID. This is mostly used as a benchmark
tier = 1
filtered_impressions = impressions_df[impressions_df["pid"].isin(client_placement_ids)]
if tier == 1:
    filtered_impressions = filtered_impressions[filtered_impressions["part_month"] == "09"]
if tier < 3:
    filtered_impressions = filtered_impressions[filtered_impressions["chosen_brand"] == client.lower()]

In [156]:
def _get_final_fixation(gaze_valid, is_fixated, id_, pred_map):
    if gaze_valid:
        return is_fixated
    if id_ in pred_map:
        return pred_map[id_]
    return False

filtered_impressions['final_fixation'] = filtered_impressions.apply(lambda row: _get_final_fixation(row['gaze_valid'], row['is_fixated'], row['id'], pred_map), axis=1)

In [157]:
filtered_impressions["true_is_iab_inview"] = filtered_impressions.apply(
    lambda x: 
    True if (
        (x["ad_technical_format"] == "out-stream") & (x["exist_viewable_1_s_threshold_50"] == True) & (x["exist_viewable_2_s_threshold_50"] == False)
    ) | (x["is_iab_inview"] == True) 
    else False, 
    axis=1
)

In [158]:
filtered_impressions["total_fixation_duration"] = filtered_impressions["total_fixation_duration"].astype("Int64")
# Normalise outliers for fixation duration (30 seconds)
filtered_impressions.loc[filtered_impressions["total_fixation_duration"] >= 30000, "total_fixation_duration"] = 30000

In [159]:
grouped_df = filtered_impressions.groupby(["pid", "hostname"]).apply(
    lambda x: pd.Series(
        {
            "impressions": x["id"].count(),
            "fixations": x.loc[x["final_fixation"], "id"].count(),
            "inview": x.loc[x["true_is_iab_inview"], "id"].count(),
            "total_fixation_duration": x["total_fixation_duration"].sum(),
            "average_fixation_duration": x["total_fixation_duration"].mean()
        }
    )
)
grouped_df = grouped_df.reset_index()

/tmp/ipykernel_11810/325538934.py:1: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  grouped_df = filtered_impressions.groupby(["pid", "hostname"]).apply(


ValueError: cannot insert hostname, already exists

In [149]:
grouped_client = client_df.groupby(["Placement Id"]).apply(
    lambda x: pd.Series(
        {
            "Imps": x["Imps"].sum(),
            "Viewability Rate %": np.average(x["Viewability Rate %"], weights=x["Imps"])
        }
    )
)
grouped_client = grouped_client.reset_index()

In [150]:
merged_df = grouped_df.merge(grouped_client, how="inner", left_on="pid", right_on="Placement Id")
merged_df = merged_df[["hostname", "Placement Id", "impressions", "fixations", "inview", "Imps", "Viewability Rate %", "average_fixation_duration", "total_fixation_duration"]]
merged_df["fixation_ratio"] = round(merged_df["fixations"] / merged_df["impressions"], 2)
merged_df["client_fixations_per_placement"] = (merged_df["fixation_ratio"] * merged_df["Imps"]).fillna(0).replace([np.inf, -np.inf], 0)

In [151]:
final_group = merged_df.groupby(["hostname"]).apply(
     lambda x: pd.Series(
         {
             "tobii_imps": x["impressions"].sum(),
             "client_imps": x["Imps"].sum(),
             "tobii_fixations": x["fixations"].sum(),
             "tobii_inview": x["inview"].sum(),
             "client_fixations_per_placement": x["client_fixations_per_placement"].sum(),
             "client_viewability_rate": np.average(x["Viewability Rate %"], weights=x["Imps"]),
             "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),
             "total_fixation_duration": x["total_fixation_duration"].sum()
         }
     )
)
final_group = final_group.reset_index()
final_group["tobii_imps"] = final_group["tobii_imps"].astype("Int64")
final_group["client_imps"] = final_group["client_imps"].astype("Int64")
final_group["tobii_fixations"] = final_group["tobii_fixations"].astype("Int64")
final_group["tobii_inview"] = final_group["tobii_inview"].astype("Int64")
final_group["tobii_inview/impression_ratio"] = round(final_group["tobii_inview"] / final_group["tobii_imps"] * 100, 2)
final_group["tobii_fixation/impression_ratio"] = round(final_group["tobii_fixations"] / final_group["tobii_imps"] * 100, 2)
final_group = final_group.fillna(0).replace([np.inf, -np.inf], 0)

/tmp/ipykernel_11810/2389419411.py:10: RuntimeWarning: invalid value encountered in scalar divide
  "average_fixation_duration": round(x["total_fixation_duration"].sum() / x["fixations"].sum(), 0),


In [152]:
final_group["client_inview"] = round(final_group["client_imps"] * final_group["client_viewability_rate"], 0).astype("Int64")
final_group["client_fixations"] = round(final_group["client_fixations_per_placement"], 0).astype("Int64")
final_group["client_fix/inview_ratio"] = round(final_group["client_fixations"] / final_group["client_inview"] * 100, 2)
final_group["client_total_fixation_duration"] = final_group["client_fixations"] * final_group["average_fixation_duration"]

def get_sample_size(num_impressions):
    if num_impressions <= 99:
        return "low"
    elif num_impressions >= 100 and num_impressions <= 199:
        return "medium"
    else:
        return "high"

final_group["sample_size"] = final_group["tobii_imps"].apply(lambda x: get_sample_size(x))

In [153]:
final_group

,hostname,tobii_imps,client_imps,tobii_fixations,tobii_inview,client_fixations_per_placement,client_viewability_rate,average_fixation_duration,total_fixation_duration,tobii_inview/impression_ratio,tobii_fixation/impression_ratio,client_inview,client_fixations,client_fix/inview_ratio,client_total_fixation_duration,sample_size
0,aftonbladet.se,1456,1570562,348,849,378274.47,0.729758,1124.0,391171.0,58.31,23.9,1146130,378274,33.0,425179976.0,high
1,klart.se,6,2493,0,1,0.00,0.464206,0.0,0.0,16.67,0.0,1157,0,0.0,0.0,low
2,svd.se,36,15342,19,26,3668.75,0.766479,6437.0,122310.0,72.22,52.78,11759,3669,31.2,23617353.0,low
3,tv.nu,11,10212,6,3,3795.00,0.629466,15.0,89.0,27.27,54.55,6428,3795,59.04,56925.0,low


In [154]:
result_df = final_group[["hostname", "tobii_imps", "client_imps", "tobii_fixations", "client_fixations", "tobii_inview", "client_inview", "client_fix/inview_ratio", "tobii_inview/impression_ratio", "client_viewability_rate", "tobii_fixation/impression_ratio", "average_fixation_duration", "total_fixation_duration", "client_total_fixation_duration", "sample_size"]]
result_df
result_df.to_excel(f"../temp/{client.lower()}_tier_{tier}_example_report.xlsx", index=False)

In [160]:
# Load tiered data
#df_1 = pd.read_excel(f"./{client.lower()}_tier_1_example_report.xlsx")
df_1 = pd.DataFrame({})
df_1["tier"] = 1
df_2 = pd.read_excel(f"../temp/{client.lower()}_tier_2_example_report.xlsx")
df_2["tier"] = 2
df_3 = pd.read_excel(f"../temp/{client.lower()}_tier_3_example_report.xlsx")
df_3["tier"] = 3
all_df = [df_1, df_2, df_3]
combined_client_df = pd.concat(all_df)
columns = list(df_2.columns)

In [161]:
def get_data_based_on_tier(client_dfs):
    new_df = pd.DataFrame(columns=columns)
    for i, df in enumerate(client_dfs):
        for index, row in df.iterrows():
            if row.tier == 1:
                if row.sample_size in ("high", "medium"):
                    new_df = new_df.append(row)
            elif row.tier == 2:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0 and row.tobii_imps >= 10:
                    if row.sample_size == "high":
                        # Degrade sample_size if lower tier
                        row.sample_size = "medium"
                        new_df = new_df.append(row)
                    elif row.sample_size == "medium":
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.25 + new_row["client_fixations"].sum() * 0.75, 0)
                        )
                        new_df = new_df.append(new_row)
                    else:
                        new_row = combined_client_df[(combined_client_df["tier"] == 3) & (combined_client_df["hostname"] == row.hostname)]
                        new_row.sample_size = "medium"
                        new_row["client_fixations"] = (
                            round(row["client_fixations"] * 0.15 + new_row["client_fixations"].sum() * 0.85, 0)
                        )
                        new_df = new_df.append(new_row)
            elif row.tier == 3:
                if new_df[new_df["hostname"] == row.hostname]["hostname"].count() == 0:
                    row["sample_size"] = "low"
                    new_df = new_df.append(row)
    return new_df

final_client_df = get_data_based_on_tier(all_df)

/tmp/ipykernel_11810/764905171.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df = new_df.append(row)
/tmp/ipykernel_11810/764905171.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row.sample_size = "medium"
/tmp/ipykernel_11810/764905171.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_row["client_fixations"] = (
/tmp/ipykernel_11810/764905171.py:27: FutureWarning: The frame.append method is 

In [162]:
final_client_df

,hostname,tobii_imps,client_imps,tobii_fixations,client_fixations,tobii_inview,client_inview,client_fix/inview_ratio,tobii_inview/impression_ratio,client_viewability_rate,tobii_fixation/impression_ratio,average_fixation_duration,total_fixation_duration,client_total_fixation_duration,sample_size,tier
0,aftonbladet.se,1456,1570562,348,378274,849,1146130,33.00,58.31,0.729758,23.90,1124,391171,425179976,medium,2
4,svd.se,5968.0,19702.0,2144.0,3962.0,3999.0,15050.0,26.67,67.01,0.763862,35.92,4651.0,9971539.0,18669114.0,medium,3
5,tv.nu,578.0,12491.0,116.0,2738.0,289.0,8122.0,31.42,50.00,0.650218,20.07,1234.0,143147.0,3149168.0,medium,3
1,bloggar.aftonbladet.se,11,79122,0,0,3,52807,0.00,27.27,0.667407,0.00,0,0,0,low,3
2,klart.se,641,4054,137,833,345,2308,36.09,53.82,0.569333,21.37,2454,336165,2044182,low,3
3,malservice.aftonbladet.se,47,5,7,1,16,3,33.33,34.04,0.600000,14.89,503,3523,503,low,3


In [163]:
def do_math(row, arg1, arg2):
    try:
        return round(row[f"{arg1}"] / row[f"{arg2}"] * 100, 2)
    except ZeroDivisionError:
        return 0

final_client_df["client_Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_inview", "client_imps"), axis=1)
final_client_df["client_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_inview"), axis=1)
final_client_df["tobii_Fixation/Inview Ratio"] = final_client_df.apply(lambda x: do_math(x, "tobii_fixations", "tobii_inview"), axis=1)
final_client_df["client_fixation/impression_ratio"] = final_client_df.apply(lambda x: do_math(x, "client_fixations", "client_imps"), axis=1)
final_client_df["Total Fixation Duration"] = final_client_df["average_fixation_duration"] * final_client_df["client_fixations"]
final_client_df = final_client_df[["hostname", "client_imps", "client_fixations", "tobii_fixation/impression_ratio", "client_inview", "client_Inview Ratio", "client_Fixation/Inview Ratio", "tobii_Fixation/Inview Ratio", "tobii_inview/impression_ratio", "Total Fixation Duration", "average_fixation_duration", "sample_size", "tier"]]
final_client_df = final_client_df.rename(
    columns={
        "client_imps": "Impressions", 
        "client_fixations": "Fixations",
        "client_inview": "Inviews",
        "client_Inview Ratio": "Inview Ratio",
        "tobii_Fixation/Inview Ratio": "Fixation/Tobii Inview Ratio",
        "client_Fixation/Inview Ratio": "Fixation/Inview Ratio",
        "average_fixation_duration": "Average Fixation Duration",
        "sample_size": "Sample Size"
    }
)
final_client_df

,hostname,Impressions,Fixations,tobii_fixation/impression_ratio,Inviews,Inview Ratio,Fixation/Inview Ratio,Fixation/Tobii Inview Ratio,tobii_inview/impression_ratio,Total Fixation Duration,Average Fixation Duration,Sample Size,tier
0,aftonbladet.se,1570562,378274,23.90,1146130,72.98,33.00,40.99,58.31,425179976,1124,medium,2
4,svd.se,19702.0,3962.0,35.92,15050.0,76.39,26.33,53.61,67.01,18427262.0,4651.0,medium,3
5,tv.nu,12491.0,2738.0,20.07,8122.0,65.02,33.71,40.14,50.00,3378692.0,1234.0,medium,3
1,bloggar.aftonbladet.se,79122,0,0.00,52807,66.74,0.00,0.00,27.27,0,0,low,3
2,klart.se,4054,833,21.37,2308,56.93,36.09,39.71,53.82,2044182,2454,low,3
3,malservice.aftonbladet.se,5,1,14.89,3,60.00,33.33,43.75,34.04,503,503,low,3


In [164]:
final_client_df = final_client_df[["hostname", "Impressions", "Inviews", "Fixations", "Fixation/Inview Ratio", "Average Fixation Duration", "Total Fixation Duration", "Sample Size"]]
final_client_df = final_client_df.sort_values(by=["hostname"])

In [165]:
comments = ["Tier 1", "Tier 2", "Tier 3"]

with pd.ExcelWriter(f"../temp/{client}_results_internal.xlsx") as writer:
    rows = 0
    spaces = 0
    for n, df in enumerate(all_df):
        pd.Series(comments[n]).to_excel(writer, sheet_name=client.title(), index=False, header=False, startrow=rows + spaces)
        df.to_excel(writer, client.title(), index=False, startrow=1 + rows + spaces)
        rows += len(df) + 2
        spaces += 2

with pd.ExcelWriter(f"../temp/{client}_results.xlsx") as writer:
    for df in [final_client_df]:
        df.to_excel(writer, sheet_name=client.title(), index=False)